Table: Scores


+-------------+---------+
| Column Name | Type    |

+-------------+---------+
| id          | int     |
| score       | decimal |
+-------------+---------+

id is the primary key (column with unique values) for this table.
Each row of this table contains the score of a game. Score is a floating point value with two decimal places.
 

Write a solution to find the rank of the scores. The ranking should be calculated according to the following rules:

The scores should be ranked from the highest to the lowest.
If there is a tie between two scores, both should have the same ranking.
After a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no holes between ranks.
Return the result table ordered by score in descending order.

The result format is in the following example.

 

Example 1:

Input: 

Scores table:

+----+-------+
| id | score |

+----+-------+
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |
+----+-------+
Output: 

+-------+------+
| score | rank |

+-------+------+
| 4.00  | 1    |
| 4.00  | 1    |
| 3.85  | 2    |
| 3.65  | 3    |
| 3.65  | 3    |
| 3.50  | 4    |
+-------+------+


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import dense_rank
from pyspark.sql.window import Window

# Initialize Spark session
spark = SparkSession.builder.appName("RankScores").getOrCreate()

# Sample data
data = [
    (1, 3.50),
    (2, 3.65),
    (3, 4.00),
    (4, 3.85),
    (5, 4.00),
    (6, 3.65)
]

# Create DataFrame
scores_df = spark.createDataFrame(data, ["id", "score"])

In [0]:
# Define the window specification
window_spec = Window.orderBy(scores_df.score.desc())

# Apply the dense_rank function
ranked_df = scores_df.withColumn("rank", dense_rank().over(window_spec))

# Sort the result by score in descending order
result_df = ranked_df.orderBy(ranked_df.score.desc())
result_df.display()

id,score,rank
3,4.0,1
5,4.0,1
4,3.85,2
2,3.65,3
6,3.65,3
1,3.5,4


In [0]:
scores_df.createOrReplaceTempView('Scores')

In [0]:
%sql
SELECT 
    score,
    dense_rank() OVER (ORDER BY score DESC) AS rank
FROM Scores
ORDER BY score DESC;


score,rank
4.0,1
4.0,1
3.85,2
3.65,3
3.65,3
3.5,4
